# Landsat Time Series
This notebook demonstrates how to preprocess Landsat 8-9 into a time series with spectralmatch. Starting from Landsat 8-9 OLI/TIRS C2 L1, clipping clouds with OmniCloudMask, masking high NDVI areas as Pseudo Invariant Features (PIFs), applying global regression Relative Radiometric Normalization, fine-tuning overlap areas with local block adjustment, and before vs after statistics.

In [18]:
# Setup
import os
import os
import importlib

import spectralmatch.match.global_regression as global_regression_mod
import spectralmatch.match.local_block_adjustment as local_block_adjustment_mod
import spectralmatch.handlers as handlers_mod

importlib.reload(global_regression_mod)
importlib.reload(local_block_adjustment_mod)
importlib.reload(handlers_mod)

from spectralmatch.match.global_regression import global_regression
from spectralmatch.match.local_block_adjustment import local_block_adjustment
from spectralmatch.handlers import merge_rasters

working_directory = os.path.join(os.getcwd(), "data_landsat")
# This script is setup to perform matching on all tif files from a folder within the working directory called "input" e.g. working_directory/input/*.tif.
print(working_directory)

/Users/kanoalindiwe/Downloads/Projects/spectralmatch/docs/examples/data_landsat


In [23]:
# Cloud clipping
from spectralmatch import (
    create_cloud_mask_with_omnicloudmask,
    post_process_raster_cloud_mask_to_vector)
from spectralmatch import write_vector

input_folder = os.path.join(working_directory, "Input")
output_folder = os.path.join(working_directory, 'Masks'); os.makedirs(output_folder, exist_ok=True)

input_image_paths_array = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(".tif")]

# for path in input_image_paths_array:
#     create_cloud_mask_with_omnicloudmask(
#         path,
#         5,
#         3,
#         8,
#         os.path.join(output_folder, f"{os.path.splitext(os.path.basename(path))[0]}_CloudMask.tif"),
#         # down_sample_m=10
#     )

input_mask_rasters = [os.path.join(output_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(".tif")] 

for path in input_mask_rasters:
    write_vector(
        post_process_raster_cloud_mask_to_vector(
            path,
            None,
            {1: 50},
            {0: 0, 1: 1, 2: 1, 3: 1}
        ),
        os.path.join(output_folder, f"{os.path.splitext(os.path.basename(path))[0]}_CloudMask.gpkg"),
    )

RasterioIOError: /Users/kanoalindiwe/Downloads/Projects/spectralmatch/docs/examples/data_landsat/Masks/Masked_20220313.tif: No such file or directory

In [ ]:
# Masking trees for isolated analysis
from spectralmatch import (
    create_ndvi_mask,
    post_process_threshold_to_vector
)

create_ndvi_mask(
    "input_image_path.tif",
    "output_image_path.tif",
    4,
    3,
)
post_process_threshold_to_vector(
    "input_image_path.tif",
    'output_vector_path.gpkg',
    0.2,
    "<=",
)

In [ ]:
# Global histogram matching
from spectralmatch import global_regression, local_block_adjustment
from spectralmatch import merge_rasters



vector_mask_path = working_directory + "/Input/Masks.gpkg"

input_folder = os.path.join(working_directory, "Input")
global_folder = os.path.join(working_directory, "Output/GlobalMatch")
local_folder = os.path.join(working_directory, "Output/LocalMatch")

input_image_paths_array = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(".tif")]

matched_global_images_paths = global_regression(
    input_image_paths_array,
    global_folder,
    custom_mean_factor = 3, # Defualt 1; 3 often works better to 'move' the spectral mean of images closer together
    custom_std_factor = 1,
    # vector_mask_path=vector_mask_path,
    debug_logs=False,
    window_size=(512, 512),
    parallel=True,
    )



In [ ]:
# Local histogram matching
global_image_paths_array = [os.path.join(f"{global_folder}/Images", f) for f in os.listdir(f"{global_folder}/Images") if f.lower().endswith(".tif")]

matched_local_images_paths = local_block_adjustment(
    global_image_paths_array,
    local_folder,
    target_blocks_per_image=100,
    projection="EPSG:6635",
    debug_logs=False,
    window_size=(512, 512),
    parallel=True,
    )

merge_rasters(
    matched_local_images_paths, # Rasters are layered with the last ones on top
    os.path.join(working_directory, "Output/LocalMatch/MatchedLocalImages.tif"),
    window_size=(512, 512),
    )

print("Done with global and local histogram matching")